In [1]:
import numpy as np
import pickle
import keras.backend as K

from keras.layers import  GlobalAveragePooling1D, Input, Activation, MaxPooling1D, BatchNormalization, Dense, Dropout, Conv1D,GlobalMaxPooling1D
from keras.layers import GRU,AveragePooling1D,CuDNNGRU
from keras.layers.merge import Concatenate
from keras.models import Model 
from keras.callbacks import EarlyStopping,ModelCheckpoint


import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [2]:
import numpy as np
alphabet = np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                     'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
	X = np.zeros(MAX_SEQ_LEN)

	for i, ch in enumerate(line[:MAX_SEQ_LEN]):
		X[i] = smi_ch_ind[ch]

	return X #.tolist()

def letter_one_hot(aa):
    one_hot = np.zeros(20)
    for idx, letter in enumerate(alphabet):
        if aa == letter:
            one_hot[idx] = 1
            return one_hot


# Convert an entire protein to one-hot representation.
def protein_one_hot(protein_sequence, MAX_SEQ_LEN):
    #  Remove non-specific AA codes (very few are actually present in this dataset)
    protein_sequence = protein_sequence.replace('B', '')
    protein_sequence = protein_sequence.replace('J', '')
    protein_sequence = protein_sequence.replace('O', '')
    protein_sequence = protein_sequence.replace('U', '')
    protein_sequence = protein_sequence.replace('X', '')
    protein_sequence = protein_sequence.replace('Z', '')
    one_hot_seq = np.zeros( (MAX_SEQ_LEN, 20))
    for idx, aa in enumerate(protein_sequence[:MAX_SEQ_LEN]):
        one_hot_seq[idx, :] = letter_one_hot(aa)
    return one_hot_seq


In [3]:
import keras
feature_len = 768
max_go_len = 256
max_seq_len = 1000

from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di



class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,  ppi_pair_file, batch_size=128):
        'Initialization' 
        self.batch_size = batch_size
        self.ppi_pair_file = ppi_pair_file
         
        self.max_seqlen = max_seq_len
        self.max_golen = max_go_len
         
        self.protein2seq = load_dict('dm2prot2seq.pkl')
        self.read_ppi()
        self.protein2onehot = {}
        self.onehot_seqs()
         
        self.on_epoch_end()
    
    def read_ppi(self):
        with open(self.ppi_pair_file, 'r') as f:
            self.ppi_pairs  =  f.readlines()
    
    def onehot_seqs(self):
        for key, value in self.protein2seq.items():
            self.protein2onehot[key] =  protein_one_hot(value, self.max_seqlen) 
            
    
   

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.ppi_pairs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.ppi_pairs))
         
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

         
        X_seq1 = np.empty((self.batch_size, self.max_seqlen,20))
        X_seq2 = np.empty((self.batch_size, self.max_seqlen,20))
        y = np.empty((self.batch_size))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '+':
                y[i] = 1
            else:
                y[i] = 0
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
             
            
            
        return [  X_seq1, X_seq2] ,  y



    def all_data(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization

         
        X_seq1 = np.empty((len(list_IDs_temp), self.max_seqlen,20))

         
        X_seq2 = np.empty((len(list_IDs_temp), self.max_seqlen,20))
        y = np.empty((len(list_IDs_temp)))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            ppi_pair = self.ppi_pairs[ID]
            p1, p2, label = ppi_pair.rstrip().split('\t')
            if label == '+':
                y[i] = 1
            else:
                y[i] = 0
            X_seq1[i] =  self.protein2onehot[p1]
            X_seq2[i] =  self.protein2onehot[p2]
           
        return [  X_seq1, X_seq2] ,  y

In [4]:
from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer


def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        # todo: check that this is correct
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class Attention(Layer):
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False,
                 **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Note: The layer has been tested with Keras 1.x
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2 - Get the attention scores
            hidden = LSTM(64, return_sequences=True)(words)
            sentence, word_scores = Attention(return_attention=True)(hidden)
        """
        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = x * K.expand_dims(a)

        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return input_shape[0], input_shape[-1]

In [5]:
from keras.layers import   Embedding
from keras.layers import  GRU, Bidirectional, CuDNNGRU, Lambda, Flatten
from keras.layers.merge import concatenate
def inception_block(input_tensor, output_size):
    """"""
    con1d_filters = int(output_size/4)
    y = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x1 = Conv1D(con1d_filters, 5, activation="relu", padding='same')(y)

    y = Conv1D(con1d_filters, 1, activation="relu", padding='valid')(input_tensor)
    x2 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(y)

    x3 = Conv1D(con1d_filters, 3, activation="relu", padding='same')(input_tensor)
    x4 = Conv1D(con1d_filters, 1, activation="relu", padding='same')(input_tensor)

    y = Concatenate()([x1, x2, x3, x4])
#     y = MaxPooling1D(2)(mix0)
    # y = AveragePooling1D()(mix0)
#     y = BatchNormalization()(y)

    return y

def create_share_model():
    con_filters = 128
    X_input = Input(shape=(max_seq_len,20))
    # text-CNN
#     卷积池化-全连接 ---拼接-全连接  -双向GRU-全连接
    cnn = Conv1D(128, 3)(X_input)
#     cnn = BatchNormalization()(cnn)
    cnn = Activation('relu')(cnn)
     
    cnn = GlobalAveragePooling1D()(cnn)
    
    
    model = Model(X_input, cnn)
    return model


def build_siamese_model():
    left_input_seq = Input(shape=(max_seq_len,20))  
    right_input_seq = Input(shape=(max_seq_len,20))
    
    siamese_a = create_share_model()
    siamese_b = create_share_model()
    

    encoded_l = siamese_a(left_input_seq)
    encoded_r = siamese_b(right_input_seq)
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
#     dense = Dense(128,activation='relu')(L1_distance)
    prediction = Dense(1,activation='sigmoid')(L1_distance)

    siamese_net = Model(inputs=[left_input_seq,right_input_seq],outputs=prediction)
    
    siamese_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return siamese_net

def build_model():
    con_filters = 128
    
    left_input_seq = Input(shape=(max_seq_len,20))  
    right_input_seq = Input(shape=(max_seq_len,20))
    
    x = inception_block(left_input_seq,con_filters )
    x = Dropout(0.3)(x)
    x_gru = Bidirectional(CuDNNGRU(64, return_sequences=True))(left_input_seq)
    x_gru = Dropout(0.3)(x_gru)
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)
    left_x_seq = Concatenate()([x_a  , x_b, x_gru_a, x_gru_b])
    left_x_seq = Dense(256, activation='relu')(left_x_seq)
     
 
    x = inception_block(right_input_seq,con_filters )
    x = Dropout(0.3)(x)
    x_gru =Bidirectional(CuDNNGRU(64, return_sequences=True))(right_input_seq)
    x_gru = Dropout(0.3)(x_gru)
    x_a = GlobalAveragePooling1D()(x)
    x_b = GlobalMaxPooling1D()(x)
    x_gru_a = GlobalAveragePooling1D()(x_gru)
    x_gru_b = GlobalMaxPooling1D()(x_gru)  
    right_x_seq = Concatenate()([x_a  , x_b, x_gru_a, x_gru_b])
    right_x_seq = Dense(256, activation='relu')(right_x_seq)
     
    x =   Concatenate()([left_x_seq, right_x_seq])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.1)(x)
  
    x = Dense(1)(x)
    output = Activation('sigmoid')(x)
    # model = Model([left_input_go, right_input_go], output)
  
    model = Model([left_input_seq, right_input_seq], output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = build_model()
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000, 20)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1000, 20)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1000, 32)     1952        input_1[0][0]                    
___________________________________________________________________________

In [6]:
from sklearn.model_selection import StratifiedKFold
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
dataset_name = 'DM2'
for rep in range(3):
    n_splits = 10
    TPRs =  np.zeros(n_splits)
    FPRs = np.zeros(n_splits)
    Precs = np.zeros(n_splits)
    ACCs = np.zeros(n_splits)
    F1s = np.zeros(n_splits)
    MCCs = np.zeros(n_splits)
    AUCs = np.zeros(n_splits)
    count = 0
    for split in range(n_splits):
        train_pairs_file = 'CV/train'+str(rep)+'-'+str(split)
        test_pairs_file = 'CV/test'+str(rep)+'-'+str(split)
        valid_pairs_file = 'CV/valid'+str(rep)+'-'+str(split)

        batch_size = 32
        train_generator = DataGenerator(   train_pairs_file,batch_size = batch_size )
        test_generator = DataGenerator(   test_pairs_file,batch_size = batch_size)
        valid_generator = DataGenerator(   valid_pairs_file,batch_size = batch_size)
         

        model = build_model()
        save_model_name = 'CV/seq'+str(rep)+'-'+str(split) + '.hdf5'
        
        earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
        save_checkpoint = ModelCheckpoint(save_model_name, save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True)

         
        # validation_data = (valid_X, valid_Y),  verbose=1,callbacks=[earlyStopping, save_checkpoint]
        hist = model.fit_generator(generator=train_generator,
                    validation_data=valid_generator,
                    epochs = 100,verbose=1,callbacks=[earlyStopping, save_checkpoint] )
         
        
        # model = load_model(save_model_name)
        model.load_weights(save_model_name)
        with open(test_pairs_file, 'r') as f:
            test_ppi_pairs  =  f.readlines()

        test_len = len(test_ppi_pairs) 
        list_IDs_temp = np.arange(test_len)

        test_x, y_test = test_generator.all_data(list_IDs_temp)

        y_pred_prob = model.predict(test_x)

       
        y_pred = (y_pred_prob > 0.5)
        auc = metrics.roc_auc_score(y_test, y_pred_prob) 
        f1 = f1_score(y_test, y_pred)
        pre = precision_score(y_test, y_pred)
        acc = accuracy_score(y_test, y_pred)

        precision, recall, _thresholds = metrics.precision_recall_curve(y_test, y_pred_prob)
        pr_auc = metrics.auc(recall, precision)
        mcc = matthews_corrcoef(y_test, y_pred)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        total=tn+fp+fn+tp
        sen = float(tp)/float(tp+fn)
        sps = float(tn)/float((tn+fp))

        tpr = float(tp)/float(tp+fn)
        fpr = float(fp)/float((tn+fp))
        print('--------------------------\n')
        print ('AUC: %f' % auc)
        print ('ACC: %f' % acc) 
        # print("PRAUC: %f" % pr_auc)
        print ('MCC : %f' % mcc)
        # print ('SEN: %f' % sen)
        # print ('SEP: %f' % sps)
        print('TPR:%f'%tpr)
        print('FPR:%f'%fpr)
        print('Pre:%f'%pre)
        print('F1:%f'%f1)
        print('--------------------------\n')
        TPRs[count] = tpr
        FPRs[count] = fpr
        Precs[count] =pre
        ACCs[count] =acc
        F1s[count] =f1
        MCCs[count] =mcc
        AUCs[count] =auc
        count += 1
    print ('mean AUC: %f' % np.mean(AUCs))
    print ('mean ACC: %f' % np.mean(ACCs)) 
    print ('mean MCC : %f' % np.mean(MCCs))
    print('mean TPR:%f'% np.mean(TPRs))
    print('mean FPR:%f'% np.mean(FPRs))
    print('mean Pre:%f'% np.mean(Precs))
    print('mean F1:%f'% np.mean(F1s))
    np.savez('dm2_seq_'+str(rep), AUCs=AUCs, ACCs=ACCs, MCCs=MCCs, TPRs = TPRs, FPRs=FPRs, Precs=Precs, F1s=F1s)

Epoch 1/100
14/14 [==============================] - 8s 560ms/step - loss: 0.7628 - acc: 0.4866 - val_loss: 0.6955 - val_acc: 0.4896
Epoch 2/100
14/14 [==============================] - 3s 181ms/step - loss: 0.6966 - acc: 0.5022 - val_loss: 0.6930 - val_acc: 0.4792
Epoch 3/100
14/14 [==============================] - 3s 182ms/step - loss: 0.6959 - acc: 0.4777 - val_loss: 0.6928 - val_acc: 0.5104
Epoch 4/100
14/14 [==============================] - 3s 185ms/step - loss: 0.6925 - acc: 0.5134 - val_loss: 0.6928 - val_acc: 0.5833
Epoch 5/100
14/14 [==============================] - 3s 183ms/step - loss: 0.6908 - acc: 0.5246 - val_loss: 0.6920 - val_acc: 0.5104
Epoch 6/100
14/14 [==============================] - 2s 179ms/step - loss: 0.7004 - acc: 0.4710 - val_loss: 0.6936 - val_acc: 0.4896
Epoch 7/100
14/14 [==============================] - 3s 183ms/step - loss: 0.6931 - acc: 0.5379 - val_loss: 0.6925 - val_acc: 0.5000
Epoch 8/100
14/14 [==============================] - 3s 180ms/step - 

/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


--------------------------

AUC: 0.500000
ACC: 0.468750
MCC : 0.000000
TPR:0.000000
FPR:0.000000
Pre:0.000000
F1:0.000000
--------------------------

Epoch 1/100
14/14 [==============================] - 7s 468ms/step - loss: 0.7246 - acc: 0.4933 - val_loss: 0.7118 - val_acc: 0.4583
Epoch 2/100
14/14 [==============================] - 2s 165ms/step - loss: 0.7031 - acc: 0.4844 - val_loss: 0.6977 - val_acc: 0.4583
Epoch 3/100
14/14 [==============================] - 3s 182ms/step - loss: 0.6950 - acc: 0.5067 - val_loss: 0.6912 - val_acc: 0.5417
Epoch 4/100
14/14 [==============================] - 3s 180ms/step - loss: 0.6909 - acc: 0.5268 - val_loss: 0.6936 - val_acc: 0.4583
Epoch 5/100
14/14 [==============================] - 2s 174ms/step - loss: 0.6895 - acc: 0.5246 - val_loss: 0.6893 - val_acc: 0.5521
Epoch 6/100
14/14 [==============================] - 3s 179ms/step - loss: 0.6745 - acc: 0.5960 - val_loss: 0.6723 - val_acc: 0.5938
Epoch 7/100
14/14 [==============================] -

Epoch 17/100
14/14 [==============================] - 2s 176ms/step - loss: 0.1567 - acc: 0.9308 - val_loss: 0.8525 - val_acc: 0.5417
Epoch 18/100
14/14 [==============================] - 2s 172ms/step - loss: 0.1089 - acc: 0.9643 - val_loss: 0.7529 - val_acc: 0.5938
Epoch 19/100
14/14 [==============================] - 2s 176ms/step - loss: 0.0686 - acc: 0.9754 - val_loss: 0.7420 - val_acc: 0.6562
Epoch 20/100
14/14 [==============================] - 2s 176ms/step - loss: 0.0535 - acc: 0.9799 - val_loss: 0.7901 - val_acc: 0.6354
Epoch 21/100
14/14 [==============================] - 2s 176ms/step - loss: 0.0395 - acc: 0.9866 - val_loss: 1.3590 - val_acc: 0.5000
--------------------------

AUC: 0.955956
ACC: 0.859375
MCC : 0.727688
TPR:0.810811
FPR:0.074074
Pre:0.937500
F1:0.869565
--------------------------

Epoch 1/100
14/14 [==============================] - 9s 629ms/step - loss: 0.7090 - acc: 0.5268 - val_loss: 0.6944 - val_acc: 0.5104
Epoch 2/100
14/14 [============================

Epoch 3/100
14/14 [==============================] - 3s 243ms/step - loss: 0.6937 - acc: 0.4821 - val_loss: 0.6938 - val_acc: 0.4583
Epoch 4/100
14/14 [==============================] - 4s 250ms/step - loss: 0.6923 - acc: 0.5246 - val_loss: 0.6897 - val_acc: 0.5521
Epoch 5/100
14/14 [==============================] - 3s 248ms/step - loss: 0.6890 - acc: 0.5246 - val_loss: 0.6847 - val_acc: 0.5521
Epoch 6/100
14/14 [==============================] - 3s 247ms/step - loss: 0.6816 - acc: 0.5446 - val_loss: 0.6788 - val_acc: 0.6146
Epoch 7/100
14/14 [==============================] - 4s 254ms/step - loss: 0.6570 - acc: 0.6094 - val_loss: 0.6558 - val_acc: 0.6146
Epoch 8/100
14/14 [==============================] - 3s 244ms/step - loss: 0.6107 - acc: 0.6629 - val_loss: 0.6677 - val_acc: 0.5833
Epoch 9/100
14/14 [==============================] - 3s 248ms/step - loss: 0.6681 - acc: 0.5714 - val_loss: 0.6398 - val_acc: 0.6562
Epoch 10/100
14/14 [==============================] - 3s 242ms/step -

...

14/14 [==============================] - 3s 250ms/step - loss: 0.4750 - acc: 0.7545 - val_loss: 0.6581 - val_acc: 0.6458
Epoch 12/100
14/14 [==============================] - 4s 256ms/step - loss: 0.5300 - acc: 0.7009 - val_loss: 0.6640 - val_acc: 0.5938
Epoch 13/100
14/14 [==============================] - 3s 244ms/step - loss: 0.4064 - acc: 0.8080 - val_loss: 0.6614 - val_acc: 0.6146
Epoch 14/100
14/14 [==============================] - 3s 245ms/step - loss: 0.3490 - acc: 0.8371 - val_loss: 0.7268 - val_acc: 0.6458
Epoch 15/100
14/14 [==============================] - 3s 247ms/step - loss: 0.3564 - acc: 0.8326 - val_loss: 0.7595 - val_acc: 0.6667
Epoch 16/100
14/14 [==============================] - 3s 250ms/step - loss: 0.3640 - acc: 0.8304 - val_loss: 0.7734 - val_acc: 0.5625
Epoch 17/100
14/14 [==============================] - 4s 252ms/step - loss: 0.2834 - acc: 0.8594 - val_loss: 0.9840 - val_acc: 0.4583
Epoch 18/100
14/14 [==============================] - 3s 249ms/step - loss:

14/14 [==============================] - 3s 233ms/step - loss: 0.0347 - acc: 0.9866 - val_loss: 0.9624 - val_acc: 0.6562
Epoch 27/100
14/14 [==============================] - 3s 198ms/step - loss: 0.0153 - acc: 1.0000 - val_loss: 0.9253 - val_acc: 0.7083
--------------------------

AUC: 0.948192
ACC: 0.890625
MCC : 0.783872
TPR:0.838710
FPR:0.060606
Pre:0.928571
F1:0.881356
--------------------------

mean AUC: 0.916099
mean ACC: 0.811506
mean MCC : 0.641756
mean TPR:0.763237
mean FPR:0.128010
mean Pre:0.855609
mean F1:0.795605
Epoch 1/100
14/14 [==============================] - 16s 1s/step - loss: 0.7323 - acc: 0.4598 - val_loss: 0.6944 - val_acc: 0.4792
Epoch 2/100
14/14 [==============================] - 4s 251ms/step - loss: 0.6939 - acc: 0.5290 - val_loss: 0.6922 - val_acc: 0.5208
Epoch 3/100
14/14 [==============================] - 3s 242ms/step - loss: 0.6977 - acc: 0.5000 - val_loss: 0.6938 - val_acc: 0.4792
Epoch 4/100
14/14 [==============================] - 4s 251ms/step - 

14/14 [==============================] - 3s 182ms/step - loss: 0.6249 - acc: 0.6786 - val_loss: 0.6364 - val_acc: 0.6667
Epoch 9/100
14/14 [==============================] - 3s 181ms/step - loss: 0.5481 - acc: 0.7165 - val_loss: 0.6595 - val_acc: 0.5417
Epoch 10/100
14/14 [==============================] - 3s 182ms/step - loss: 0.5949 - acc: 0.6585 - val_loss: 0.5864 - val_acc: 0.7188
Epoch 11/100
14/14 [==============================] - 3s 183ms/step - loss: 0.4562 - acc: 0.8326 - val_loss: 0.5575 - val_acc: 0.7083
Epoch 12/100
14/14 [==============================] - 3s 237ms/step - loss: 0.3030 - acc: 0.8772 - val_loss: 0.6355 - val_acc: 0.6771
Epoch 13/100
14/14 [==============================] - 4s 251ms/step - loss: 0.2190 - acc: 0.9152 - val_loss: 0.7880 - val_acc: 0.6562
Epoch 14/100
14/14 [==============================] - 3s 238ms/step - loss: 0.2268 - acc: 0.8996 - val_loss: 0.9671 - val_acc: 0.6458
Epoch 15/100
14/14 [==============================] - 4s 254ms/step - loss: 

14/14 [==============================] - 3s 179ms/step - loss: 0.7041 - acc: 0.4955 - val_loss: 0.6921 - val_acc: 0.5625
Epoch 3/100
14/14 [==============================] - 3s 182ms/step - loss: 0.6976 - acc: 0.4866 - val_loss: 0.6932 - val_acc: 0.4792
Epoch 4/100
14/14 [==============================] - 3s 202ms/step - loss: 0.6913 - acc: 0.5134 - val_loss: 0.6904 - val_acc: 0.5417
Epoch 5/100
14/14 [==============================] - 4s 254ms/step - loss: 0.6853 - acc: 0.5312 - val_loss: 0.6844 - val_acc: 0.6042
Epoch 6/100
14/14 [==============================] - 3s 238ms/step - loss: 0.6869 - acc: 0.5603 - val_loss: 0.6826 - val_acc: 0.5938
Epoch 7/100
14/14 [==============================] - 3s 195ms/step - loss: 0.6590 - acc: 0.6071 - val_loss: 0.6647 - val_acc: 0.6250
Epoch 8/100
14/14 [==============================] - 3s 185ms/step - loss: 0.6118 - acc: 0.6607 - val_loss: 0.6523 - val_acc: 0.5625
Epoch 9/100
14/14 [==============================] - 3s 184ms/step - loss: 0.5252

14/14 [==============================] - 4s 251ms/step - loss: 0.3511 - acc: 0.8616 - val_loss: 0.9505 - val_acc: 0.6354
Epoch 23/100
14/14 [==============================] - 4s 259ms/step - loss: 0.1591 - acc: 0.9375 - val_loss: 0.9541 - val_acc: 0.5938
Epoch 24/100
14/14 [==============================] - 3s 244ms/step - loss: 0.0450 - acc: 0.9911 - val_loss: 0.9063 - val_acc: 0.6250
Epoch 25/100
14/14 [==============================] - 4s 255ms/step - loss: 0.0455 - acc: 0.9911 - val_loss: 0.9594 - val_acc: 0.6354
Epoch 26/100
14/14 [==============================] - 4s 256ms/step - loss: 0.0177 - acc: 0.9978 - val_loss: 1.1352 - val_acc: 0.6250
--------------------------

AUC: 0.957031
ACC: 0.906250
MCC : 0.812500
TPR:0.906250
FPR:0.093750
Pre:0.906250
F1:0.906250
--------------------------

Epoch 1/100
14/14 [==============================] - 20s 1s/step - loss: 0.7463 - acc: 0.4621 - val_loss: 0.6955 - val_acc: 0.4688
Epoch 2/100
14/14 [==============================] - 4s 250ms/

In [8]:
import numpy as np 
results1 =   np.load( 'dm2_seq_0.npz')
results2 =   np.load( 'dm2_seq_1.npz')
results3 =   np.load( 'dm2_seq_2.npz')
print ('mean AUC: %f' %  ( (np.mean( results1[ 'AUCs' ] )  + np.mean(  results2[ 'AUCs' ] )  + np.mean(results3[ 'AUCs' ]))/3     ) )
print ('mean ACC: %f' %   ( (np.mean( results1[ 'ACCs' ] )  + np.mean(  results2[ 'ACCs' ] )  + np.mean(results3[ 'ACCs' ]))/3) )
print ('mean MCC : %f' %  (  (np.mean( results1[ 'MCCs' ] )  + np.mean(  results2[ 'MCCs' ] )  + np.mean(results3[ 'MCCs' ])     )/3))
print('mean TPR:%f'%    ((np.mean( results1[ 'TPRs' ] )  + np.mean(  results2[ 'TPRs' ] )  + np.mean(results3[ 'TPRs' ])     )/3))
print('mean FPR:%f'%   ( (np.mean( results1[ 'FPRs' ] )  + np.mean(  results2[ 'FPRs' ] )  + np.mean(results3[ 'FPRs' ])     )/3))
print('mean Pre:%f'%    ((np.mean( results1[ 'Precs' ] )  + np.mean(  results2[ 'Precs' ] )  + np.mean(results3[ 'Precs' ])     )/3))
print('mean F1:%f'%    ((np.mean( results1[ 'F1s' ] )  + np.mean(  results2[ 'F1s' ] )  + np.mean(results3[ 'F1s' ])     )/3))

mean AUC: 0.924124
mean ACC: 0.840041
mean MCC : 0.689123
mean TPR:0.799908
mean FPR:0.111427
mean Pre:0.844517
mean F1:0.816189
